In [1]:
import pandas as pd
from joblib import dump
from sklearn.linear_model import Lasso, LassoCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import numpy as np

C:\Users\86188\AppData\Local\Temp\ipykernel_17836\1168571455.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
genes_train = pd.read_csv("https://cs307.org/lab-05/data/genes-train.csv")
# create X and y for train
X_train = genes_train.drop("y", axis=1)
y_train = genes_train["y"]

In [3]:
X_train

,lvfyodfjhd,chlxjpxmff,otpruhfgcr,oehlcawgpf,qoizufnzxh,zkyigilmjr,qjhqeatvex,hlvjxolgkj,lrwdgjvoau,dwkbxacpab,...,baivhgmwgj,forscbxhfb,qqoqizjjqj,deynskqfbg,effadqkxyr,eofvzakuue,wismhvmnxb,cvjwglhnga,jsdmcbejne,cvoxpagxgt
0,1.523299,19.404329,0.003386,1.926715,1.301010,2.050354,4.251543,2.212030,5.086263,5.486965,...,1.336388,1.971657,73.573000,2.698167,10.228457,447.017647,1.845999,0.656854,4.641398,0.781940
1,2.125917,79.447265,0.098064,1.756492,88.349079,0.404541,4.361385,110.207743,4.232306,5.335660,...,3.751518,1.344905,10.645052,0.410613,9.216523,78.347809,0.224902,1.567216,8.906766,0.105189
2,2.301169,89.695207,0.099854,0.459814,65.344218,0.106759,3.194797,134.950376,3.997683,5.600865,...,0.628325,3.978908,31.431066,1.687640,9.344670,23.715733,3.062889,0.000000,1.210128,0.747997
3,1.748618,49.998491,0.067186,0.965176,33.704983,4.959406,3.285966,2.696591,2.715285,5.333225,...,5.088898,4.466364,56.652506,1.344890,8.942019,127.498795,1.143567,1.933546,6.783099,1.120372
4,1.666441,13.239583,0.109528,0.000000,97.397923,2.208333,3.111847,22.831821,6.380585,5.741707,...,2.623644,2.213965,83.047111,0.148668,9.313350,49.571299,1.032066,0.000000,29.263633,0.296164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,2.117729,6.385713,0.109704,0.966220,206.352017,0.217017,5.701002,257.074823,9.586723,5.351280,...,2.143226,0.430624,246.244494,0.033559,11.104503,36.874867,2.022519,0.582008,0.586234,0.805512
896,1.921916,4.848924,0.049569,2.332017,11.699854,0.404328,4.555099,121.503267,2.118825,5.541698,...,2.558042,0.000000,59.804792,2.281729,10.910872,166.735774,2.183504,0.000000,24.536640,0.342456
897,1.714503,35.534482,0.070890,3.021258,26.574131,2.068426,6.185746,30.240624,2.288780,5.539048,...,3.157934,0.000000,0.276544,4.807311,11.258259,236.580225,0.466819,1.777398,5.399876,0.589704
898,1.850003,13.045160,0.081111,0.087547,3.688737,2.051105,4.495955,41.987975,8.012388,5.580206,...,3.894236,1.470798,23.657785,4.598416,10.353938,4.010867,0.726227,0.000000,1.114646,0.340193


In [3]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
import numpy as np

# 定义包含特征工程步骤和Lasso回归的管道
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # 数据标准化
    ('poly', PolynomialFeatures()),  # 生成多项式特征
    ('lasso', Lasso(max_iter=10000))  # Lasso回归
])

# 设置参数网格，包括Lasso的alpha参数和多项式特征的度数
param_grid = {
    'poly__degree': [1, 2],  # 尝试1阶、2阶和3阶多项式特征
    'lasso__alpha': np.logspace(-3, 1, 100),  # Lasso的alpha参数范围
    # 可以根据需要添加其他参数
}

# 创建GridSearchCV实例
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1)

# 在训练数据上执行网格搜索
grid_search.fit(X_train, y_train)

# 获取最佳参数和对应的性能指标
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# 输出最佳参数
print(f"Best parameters: {best_params}")

# 输出最佳模型的RMSE
best_rmse = np.sqrt(-best_score)
print(f"Best RMSE: {best_rmse}")


Fitting 5 folds for each of 200 candidates, totalling 1000 fits
Best parameters: {'lasso__alpha': 0.01788649529057435, 'poly__degree': 1}
Best RMSE: 0.32235601585695545


In [4]:
dump(grid_search, "gene-expression.joblib")

['gene-expression.joblib']